In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import glob, os, sys
import pickle

In [10]:
from keras import backend as K
from keras.preprocessing import image
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers

from sklearn.model_selection import train_test_split

from keras.applications.resnet50 import ResNet50 as resnet
from keras.applications.resnet50 import preprocess_input as resnet_pp

from keras.optimizers import Adam

In [ ]:
main_path = r'./POETdataset/PascalImages/'
classes = ['aeroplane', 'boat', 'dog', 'bicycle', 'cat', 'cow', 'diningtable', 'horse', 'motorbike','sofa']
IMG_SIZE = 224

In [3]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
def load_object(filename):
    with open(filename, 'rb') as fp:
        return pickle.load(fp)

In [4]:
# Build a model by adding preprocessing before the pretrained CNN
def get_feature_extraction_model():
    cnn_object, pp_function, img_size = _get_pretrained_model()
    model = keras.models.Sequential()
    cnn_model = cnn_object(weights='imagenet', include_top=False, pooling='max')
    model.add(keras.layers.Lambda(pp_function, name='preprocessing', input_shape=(img_size, img_size, 3)))
    model.add(cnn_model)
    return model

# Unpacking information from the models dictionary
def _get_pretrained_model():
    cnn_object = resnet
    pp_function = resnet_pp
    img_size = IMG_SIZE
    return cnn_object, pp_function, img_size

def get_features(files, model):
    # Load images based on the size of the Lambda layer 
    # provided as the first layer before the pretrained CNN
    x = np.array([image.img_to_array(image.load_img(file, target_size=(model.layers[0].input_shape[1], model.layers[0].input_shape[1]))) for file in files])
    return model.predict(x, verbose=1)

In [5]:
model = get_feature_extraction_model()

In [ ]:
classes = [word + '*' for word in classes]
files_list = [glob.glob(main_path + class_) for class_ in classes]

for files in files_list:
    assert len(files) > 0
    
new_dir = './features'
if not os.path.exists(new_dir):
    os.makedirs(new_dir)

files_dict = {class_name.replace('*', ''): class_files for class_name, class_files in zip(classes, files_list)}
for class_number, (class_name, files) in enumerate(files_dict.items()):
    print(class_name)
    features = get_features(files, model)
    print(features.shape)
    save_object(features, new_dir + os.path.sep + class_name+'_features.pkl')

aeroplane
666/666 [==============================] - 3s 4ms/step
(666, 2048)
boat


In [13]:
X = []
y = []
for class_number, (class_name, files) in enumerate(files_dict.items()):
    features = load_object(class_name + '_features.pkl').reshape(-1,2048)
    print(features.shape)
    X.append(features)
    y.append([class_number] * features.shape[0])
    
X = np.concatenate(X)
y = np.concatenate(y)
y = keras.utils.to_categorical(y)
X.shape, y.shape

FileNotFoundError: [Errno 2] No such file or directory: './features/aeroplane*_features.pkl'

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5643, 2048), (5643, 10), (627, 2048), (627, 10))

In [11]:
dense_model = Sequential()
# model.add(Flatten(input_shape=(1,1,2048)))
dense_model.add(Dense(units=128, activation='relu', input_shape=(2048,)))
dense_model.add(Dropout(0.25))
dense_model.add(Dense(units=10, activation='softmax'))

dense_model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy'])
history = dense_model.fit(X_train,y_train,
                    batch_size = 32,
                    epochs = 100,
                    validation_data=(X_test, y_test))

Train on 5643 samples, validate on 627 samples
Epoch 1/100
5643/5643 [==============================] - 1s 221us/step - loss: 2.5196 - acc: 0.6015 - val_loss: 0.7523 - val_acc: 0.7879
Epoch 2/100
5643/5643 [==============================] - 0s 80us/step - loss: 0.8148 - acc: 0.7399 - val_loss: 0.6526 - val_acc: 0.8054
Epoch 3/100
5643/5643 [==============================] - 0s 84us/step - loss: 0.7043 - acc: 0.7647 - val_loss: 0.6460 - val_acc: 0.8150
Epoch 4/100
5643/5643 [==============================] - 0s 86us/step - loss: 0.6093 - acc: 0.7962 - val_loss: 0.6154 - val_acc: 0.8134
Epoch 5/100
5643/5643 [==============================] - 0s 79us/step - loss: 0.5412 - acc: 0.8207 - val_loss: 0.5007 - val_acc: 0.8517
Epoch 6/100
5643/5643 [==============================] - 0s 82us/step - loss: 0.5174 - acc: 0.8293 - val_loss: 0.6412 - val_acc: 0.8278
Epoch 7/100
5643/5643 [==============================] - 0s 80us/step - loss: 0.4856 - acc: 0.8288 - val_loss: 0.5267 - val_acc: 0.8469


5643/5643 [==============================] - 0s 81us/step - loss: 0.1645 - acc: 0.9327 - val_loss: 0.9190 - val_acc: 0.8485
Epoch 61/100
5643/5643 [==============================] - 0s 86us/step - loss: 0.1703 - acc: 0.9293 - val_loss: 0.9446 - val_acc: 0.8485
Epoch 62/100
5643/5643 [==============================] - 0s 82us/step - loss: 0.1720 - acc: 0.9311 - val_loss: 1.0382 - val_acc: 0.8341
Epoch 63/100
5643/5643 [==============================] - 0s 86us/step - loss: 0.1700 - acc: 0.9318 - val_loss: 0.7902 - val_acc: 0.8325
Epoch 64/100
5643/5643 [==============================] - 0s 85us/step - loss: 0.1643 - acc: 0.9309 - val_loss: 0.9111 - val_acc: 0.8421
Epoch 65/100
5643/5643 [==============================] - 0s 86us/step - loss: 0.1847 - acc: 0.9254 - val_loss: 0.8740 - val_acc: 0.8421
Epoch 66/100
5643/5643 [==============================] - 0s 81us/step - loss: 0.1760 - acc: 0.9259 - val_loss: 0.9229 - val_acc: 0.8501
Epoch 67/100
5643/5643 [==============================